## In this note book we will implement two different mutation strategies.
###  We will use the heuristic initialization created before, where the value of the initialization_percentage equals to 0.2 during the experiments of these new mutation strategies.

In [1]:
using CSV
using DataFrames
using StringEncodings
using RDatasets 
using BenchmarkTools
using Distributions
using Dates
import Dates
using SparseArrays, SharedArrays
using Distributed
using Base.Threads
using StatsBase
using LinearAlgebra

## read_data
The function belows reads each file, a is the path of data, data is a output matrix

In [2]:

function read_data(a)
    f=open(a,"r")
    s=StringDecoder(f,"LATIN1", "UTF-8")
    data= CSV.read(s)
    close(s)
    close(f)
    return data
end
    

read_data (generic function with 1 method)

## create_features
This function takes the path of two files: features of functions after the preprocessing (functionswithdomains), and the connection between the controllers. 
<br> It returns several matrix analyzed and created from these set of data.
<br>
<br> The output of this function are:
* <b>G</b> is a matrix of dimension n by m. n is the number of functions and m equals to the number of controllers. It represents the adequacy degree for each function to each controller. These data are taken from the functionWithDomain file. 
* <b>id1 and id2</b> represent respectively the index of the functions that already allocated and not allocated.
The id of functions that have one choice in DejaAllouees attribute of functionWithDomain are added to id1 and the remains functions are allocated to id2. In another way:
<br> id1 has length equals to number of allocated functions.
<br> id2 has length equals to the number of non allocated functions.
* <b>type_ctr_connection</b> is a matrix of dimension a by a, where a is the number of controllers. This Matrix represents the type of connection between the controllers. The value of type_ctr_connection[i,j] can be 0,1,2 and 3. 
<br>&nbsp;&nbsp;    0: controllers i and j can not be connected.
<br>&nbsp;&nbsp;    1: controller i can only transmit data to controller j (first way of communication).
<br>&nbsp;&nbsp;    2: controller i can only receive data from controller j (second way of communication).
<br>&nbsp;&nbsp;    3: controller i can be connected in both ways to controller j. 
This array is created in depending on the file contains the communication values between the controllers (connection_ctr).
* <b>remain_connection_ctr</b> is a matrix of dimension a by a, where a is the number of controllers. It contains the remaining capacity communication between the controllers. The calculation of this matrix is done by 2 steps:
<br>&nbsp;&nbsp;    1. By calculating the communication values between the functions assigned to different controllers. 
<br>&nbsp;&nbsp;    2. Substracting connection_ctr from the matrix calculated in the first step.
* <b>output_f,output_weight</b> are two arrays of dimensional n by k, where n is the number of functions and k is the maximum number of functions that a function is connected to them by the first way of communication (output connections).
<br>&nbsp;&nbsp;    Each row i in output_f represents id of functions that received data from function i.
<br>&nbsp;&nbsp;    output_weight contains the communication values of the communication represented in output_f. 
<br>&nbsp; These two arrays are created using outputN1 and PoidsOutut1 columns existed in functionWithDomain file. 
* <b>input_f,input_weight</b> are as the two previous array, but they represent the second way of communication instead of the first one for each function. Also these two matrix are created using InputN1 and PoidsInput1 features existed in functionWithDomain file. 
* <b> nb_connections</b> is a vector of length equals to number of functions non allocated and represents the number of connections for each function

In [3]:
function create_features(m,q)
    F1 = read_data(m); # FunctionsWithDomains
    connection_ctr1 = read_data(q); #controllers_connection

    "convert connection controllers to 2d array"
    connection_ctr = connection_ctr1[1:size(connection_ctr1,1),2:size(connection_ctr1,2)];
    connection_ctr =convert(Matrix,connection_ctr);
    controllers = String.(names(connection_ctr1));  # get the name vector of the controllers
    controllers = controllers[2:length(controllers)];
    #
    #
    nb_ctr = length(controllers);  "nb_ctr: number of controllers" 
    choixprefere = F1.ChoixPrefere;  "the attribute Choix prefere (contains numbers)"
    nb_f = length(choixprefere);  "nb_f: number of functions"
    allowed_f = F1.DejaAllouee; "get DejaAllouee attribute (contains name of controllers)"
    
    "creat connection matrix 
    output1 is vector of length nb_f represents the output connections for each function
    weights_output is the values of the output connections
    input1 is vector of length nb_f represents the input connections for each function
    weights_input is the values of the input connections
    temp is a vector of length nb_f represents the number of connections for each function"
    
    output1 =  F1.outputN1; 
    weights_output = F1.PoidsOutputN1; 
    input1 = F1.inputN1; 
    weights_input = F1.PoidsInuputN1;
    max_out = 0;
    max_in = 0;
    temp = zeros(Int,nb_f);
    
    for i in 1:nb_f
        "calculate maximum of out connections"
        inputs = output1[i];
        if length(inputs)!=2  "output1[i] different from [], it is not empty"
            inputs = inputs[2:length(inputs)-1]; "to remove [] from output[i]"
            inputs = split(inputs,";");
            inputs = [parse(Int, x) for x in inputs];  "convert from string to int"
            temp[i] += length(inputs);   "add output connections"
            if(length(inputs)>max_out)
                max_out = length(inputs);
            end
        end
        "calculate maximum of input connections"
        inputs = input1[i];
        if length(inputs)!=2  "input1[i] is not empty" 
            inputs = inputs[2:length(inputs)-1];
            inputs = split(inputs,";");
            inputs = [parse(Int, x) for x in inputs];
            temp[i] += length(inputs);   "add input connections"
            if(length(inputs)>max_in)
                max_in = length(inputs);
            end
        end
    end
    
    output_f =  zeros(Int,nb_f,max_out);  "create output_f" 
    output_weight = zeros(Int,nb_f,max_out); "create output_weight"
    input_f = zeros(Int,nb_f,max_in);
    input_weight = zeros(Int,nb_f,max_in);
    #
 "this for loop is to determine the functions connected in both ways to each function and the weights of these connections
     first one is for output connections
     second on is for input connections"
    for i in 1:nb_f
        "output connections"
        inputs = output1[i];
        weight = weights_output[i];
        if length(inputs)!=2
            inputs = inputs[2:length(inputs)-1];
            inputs = split(inputs,";");
            inputs = [parse(Int, x) for x in inputs];
            weight = weight[2:length(weight)-1];
            weight = split(weight,";");
            weight = [parse(Int, x) for x in weight];

            for j in 1:length(inputs)
                output_f[i,j] = inputs[j]+1;
                output_weight[i,j] = weight[j];
            end
        end
        "input connections"
        inputs = input1[i];
        weight = weights_input[i];
        if length(inputs)!=2
            inputs = inputs[2:length(inputs)-1];
            inputs = split(inputs,";");
            inputs = [parse(Int, x) for x in inputs];
            weight = weight[2:length(weight)-1];
            weight = split(weight,";");
            weight = [parse(Int, x) for x in weight];
            for j in 1:length(inputs)
                input_f[i,j] = inputs[j]+1;
                input_weight[i,j] = weight[j];
            end
        end
    end
    #
    #
    G = zeros(Int,nb_f,nb_ctr);
    id1 = zeros(Int,0);
    id2 = zeros(Int,0);
    for i in 1:nb_f 
        groups = allowed_f[i];   "get the name of controllers allowod to  function i"
        choix = choixprefere[i]; "get the degrees of controllers allowed to  function i"
        choix = choix[2:length(choix)-1];
        choix = split(choix,";");
        choix = [parse(Int, x) for x in choix];
                        
        groups = groups[2:length(groups)-1];
        groups = split(groups,";");
        if (length(choix)==1)  "choix = a ( one number),  function i is allocated"
            id1 = append!(id1,i);
            group = findall(x-> x==groups[1],controllers); "find the index of controller where function i is allocated"
            G[i,group] = choix; 
        "if the function i is not allocated"
        else  
            for j in 1:length(choix)
                "find the index of each controller allowed to function i  and add its degree G[i,:]"
                index_weight = findall(x-> x==groups[j],controllers);
                G[i,index_weight[1]] = choix[j];
            end
            id2 = append!(id2,i);
        end
    end
    
    "connection_G1 is a matrix of dimension nb_ctr by nb_ctr. 
    It contains the values of connections between the allocated functions."
    
    connection_G1 = zeros(Int,nb_ctr,nb_ctr);
    # connection between G1
    
    "the first for loop is to pass through all the functions
    the first if is to verify that the function is allocated
    second for loop is to pass through all the output connections of the allocated function
    second if is to verify that the connected function to function i is not zero and allocated"
    for i in 1:nb_f
        if (length(findall(x-> x!=0,G[i,1:nb_ctr]))==1)
            for j in  1:size(output_f,2) 
                if (output_weight[i,j]!=0) && (length(findall(x-> x!=0,G[output_f[i,j],1:nb_ctr]))==1) 
                        a = findall(x-> x!=0,G[i,1:nb_ctr]); "find the group of function i" 
                        b = findall(x-> x!=0,G[output_f[i,j],1:nb_ctr]); "find the group of connected function"
                        if(a[1]!=b[1])  "if the 2 functions are not in the same controller"
                            connection_G1[a[1],b[1]] += output_weight[i,j];
                        end
                end
            end
        end
    end
    
    "calculate the remain capacity connections between the controllers"
    remain_connection_ctr = connection_ctr - connection_G1;
  
    type_ctr_connection = zeros(Int,nb_ctr,nb_ctr);
    for i in 1:nb_ctr
        for j in 1:nb_ctr
            if((connection_ctr[i,j] !=0) && (connection_ctr[j,i] !=0))
                type_ctr_connection[i,j] = type_ctr_connection[j,i] = 3;
            elseif ((connection_ctr[i,j] !=0) && (connection_ctr[j,i] ==0))
                    type_ctr_connection[i,j] = 1;
            elseif ((connection_ctr[i,j] ==0) && (connection_ctr[j,i] !=0))
                    type_ctr_connection[i,j] = 2;
            else
                    type_ctr_connection[i,j]  = 0;
            end
        end
    end
    nb_connections = temp[id2];
    return G,id1,id2,type_ctr_connection,remain_connection_ctr,output_f,output_weight,input_f,input_weight,nb_connections;
end

create_features (generic function with 1 method)

## evaluation
<br>This function take as input path of choixprefere column from functionwithDomain file, population(pop), the output of creat_features function and the upper bound of the study case (up_bound)
<br> It calculates the fitness function of all the individuals and returns:
* <b> y:</b> vector of length equals to population size, it contains the result of the fitness function.
* <b> best_ind:</b> contains the maximum of y, values of violate function and capacity for the individual that has the maximum
* <b>best_individual:</b> the best individual that has the maximum of y

In [4]:

function evaluation(G,pop,type_ctr_connection,remain_connection_ctr,up_bound,output_f,output_weight,
                    input_f,input_weight)
    y = zeros(collect(size(pop))[1]);
    y1 = zeros(Int,collect(size(pop))[1]);
    "nb_ctr is number of controllers
    violate_f represents the number of functions that can not do all their connections 
    without the violation of the connection constraints between the controllers in each individual .It has length
    equals to population size.
    violate_capacity represents the values of connections betweenn the controllers
    that exceed the limit in each individual"
    
    nb_ctr = collect(size(remain_connection_ctr))[1];
    violate_f = zeros(Int,collect(size(pop))[1]);
    violate_capacity = zeros(Int,collect(size(pop))[1]);
    
    "first for loop to visit each individual"
    for i in 1:size(pop,1) 
        capacity_ctr = zeros(Int,nb_ctr,nb_ctr); "connection capacity between the functions for individual i"
        "second fo loop to visit each function in each individual"
        for j in 1:size(pop,2)  
            "verify the output connections"
            l = 0;
            if(length(findall(x-> x!=0,G[j,:]))!=1)  "if the function is not allocated"
                inputs = output_f[j,:]; 
                weight = output_weight[j,:];
                for k in 1:length(inputs) "visit the connected functions to function j"
                    if(weight[k]==0)  "means no more connection or there is no connection"
                        break;
                    elseif (pop[i,j]!=pop[i,inputs[k]]) "if j and the connected f are in different controllers"
                        "pop[i,j]: controller of function j
                         pop[i,inputs[k]] controller of function connected to j"
                        
                        capacity_ctr[pop[i,j],pop[i,inputs[k]]] += weight[k];  
                            
                        "the if below is to verify that the 2 functions are in 2 controllers 
                        can be connected in one direction"
                            
                        if((type_ctr_connection[pop[i,j],pop[i,inputs[k]]] == 2) || 
                                (type_ctr_connection[pop[i,j],pop[i,inputs[k]]]== 0))
                                    if(l==0)  "function j until now has not violated the 
                                                connectivty with any of its output connected functions"
                                        l = 1;
                                        violate_f[i] += 1;
                                    end
                            end
                    end   
                end
        
                "verify function j not allocated  can receive data from functions already allocated"
           
                inputs = input_f[j,:];
                weight = input_weight[j,:];
                for k in 1:length(inputs)
                    if (weight[k]==0)
                        break;
                    "if the function that send data to function j is allocated"
                    elseif(length(findall(x-> x!=0,G[inputs[k],:]))==1) 
                            if(pop[i,j]!=pop[i,inputs[k]]) "if the functions are not in the same group"
                                capacity_ctr[pop[i,inputs[k]],pop[i,j]] += weight[k];
                                if((type_ctr_connection[pop[i,j],pop[i,inputs[k]]] == 1) ||
                                    (type_ctr_connection[pop[i,j],pop[i,inputs[k]]]== 0))
                                    "if function j has not violated any constraint of connectivity yet"
                                    if(l==0)
                                        l = 1;
                                        violate_f[i] += 1;
                                    end
                                end
                            end

                    end
                end
            end
            #
            "if function j respect all the connectivity constraints"
            if(l==0)
                y[i] += G[j,pop[i,j]];
            end
        end 
        "end of second loop"  
        "calculate the remaining capacity between the controllers"
        capacity_ctr = remain_connection_ctr - capacity_ctr;
        for k in 1:nb_ctr
            for z in 1:nb_ctr
                "verify if any remain capacity is negative if initially this capacity is not zero"
                if(capacity_ctr[k,z]<0) && (remain_connection_ctr[k,z]!=0)
                    violate_capacity[i] += -capacity_ctr[k,z]
                end
            end
        end
    end
    y1 = y;
    y = y./up_bound;
    y = y - violate_f./(maximum(violate_f)+1);
    y = y - violate_capacity./(maximum(violate_capacity)+1);
    best_indice = argmax(y);
    best_ind = zeros(Int,3);
    best_ind[1] = y1[best_indice];
    best_ind[2] = violate_f[best_indice];
    best_ind[3] = violate_capacity[best_indice];
    return y,best_ind, pop[best_indice,:]
end

evaluation (generic function with 1 method)

## heuristic_initialization

In [21]:
function heuristic_initialization(G,population_size,initialization_perc,id2,nb_connections)
    pop = zeros(Int,population_size,collect(size(G))[1]);
    " e is the number of functions allocate to the controllers that have the highest degree between 
      their available controllers
    idx represents the indexes of these functions. These indexes are gotten by decreasing
    order sort of the functions based on their number of connections"
    e = initialization_perc*length(id2);
    e = convert(Int64, round(e, digits=0));
    idx = sortperm(nb_connections)[1:e]; 
    for j in 1:population_size
        for i in 1:e
            a = argmax(G[id2[idx[i]],:]);
            pop[j,id2[idx[i]]] = a[1];
        end
        for i in 1:size(G,1)
            c = findall(x-> x==i,id2[idx]);
            if(length(c)==0)
                b = findall(x-> x!=0, G[i,:]);
                id = rand(1:length(b));
                pop[j,i] = b[id[1]];
            end
        end
    end
    return pop;
end
        

heuristic_initialization (generic function with 1 method)

## selection

In [6]:
function selection(pop,perc_selection,y)
    a = perc_selection*length(y);
    a = convert(Int64, round(a, digits=0));
    select_pop = zeros(Int,a,collect(size(pop))[2]);
    idx = sortperm(-y)[1:a];
    for i in 1:length(idx)
        select_pop[i,:] = pop[idx[i],:];
    end  
    return select_pop;
end

selection (generic function with 1 method)

## Crossover

In [7]:
function crossover(select_pop,remain_individuals,cross_proba)
    cross_pop = zeros(Int,remain_individuals,collect(size(select_pop))[2]);
    i =1;
    best_ind = 0.25*size(select_pop,1);
    best_ind = convert(Int64, round(best_ind, digits=0));
    while(i<remain_individuals)
        a = rand(1:best_ind);
        parent_1 = select_pop[a,:]
        a = rand(best_ind+1:collect(size(select_pop))[1]);
        parent_2 = select_pop[a,:]
        for j in 1:length(parent_2)
            if (rand(Uniform(0, 1))>cross_proba)
                cross_pop[i,j] = parent_1[j];
                cross_pop[i+1,j] = parent_2[j];
            else
                cross_pop[i,j] = parent_2[j];
                cross_pop[i+1,j] = parent_1[j];
            end
        end
        i += 2;
    end
    if (remain_individuals%2!=0)
        a = rand(1:best_ind);
        parent_1 = select_pop[a,:]
        a = rand(best_ind+1:collect(size(select_pop))[1]);
        parent_2 = select_pop[a,:]
        for j in 1:length(parent_2)
            if (rand(Uniform(0, 1))>cross_proba)
                cross_pop[remain_individuals,j] = parent_1[j];
            else
                cross_pop[remain_individuals,j] = parent_2[j];
            end
        end
    end
    new_pop = [select_pop;cross_pop];
    return new_pop;
end

crossover (generic function with 1 method)

## Mutation

In [29]:
function mutation(pop,rate_perc,mut_proba,G,id2)
    rate = rate_perc*length(id2);
    rate = convert(Int64, round(rate, digits=0));
    for i in 1:size(pop,1)
        if (rand(Uniform(0, 1)) < mut_proba)
            c = sample(1:length(id2),rate, replace = false);
            for j in 1:rate
                b = findall(x-> x!=0,G[id2[c[j]],:]);
                a = rand(1:length(b));
                while (b[a]==pop[i,id2[c[j]]])
                        a = rand(1:length(b));
                end
                pop[i,id2[c[j]]] = b[a];
            end
        end
    end
    return pop;
end

mutation (generic function with 1 method)

## mutation_increased_weight
In mutation strategy below, the function that has higher number of connections, it will have higher probabulity to be a mutated gene.

In [23]:

function mutation_increased_weight(pop,rate_perc,mut_proba,G,id2,nb_connections)
    weights = nb_connections ./ sum(nb_connections);
    rate = rate_perc*length(id2);
    rate = convert(Int64, round(rate, digits=0));
    for i in 1:size(pop,1)
        if (rand(Uniform(0, 1)) < mut_proba)
            c = sample(1:length(id2),Weights(weights),rate, replace = false);
            for j in 1:rate
                b = findall(x-> x!=0,G[id2[c[j]],:]);
                a = rand(1:length(b));
                while (b[a]==pop[i,id2[c[j]]])
                        a = rand(1:length(b));
                end
                pop[i,id2[c[j]]] = b[a];
            end
        end
    end
    return pop;
end

mutation_increased_weight (generic function with 1 method)

## mutation_decreased_weight
In mutation strategy below, the function that has lower number of connections, it will have higher probabulity to be a mutated gene.

In [24]:

function mutation_decreased_weight(pop,rate_perc,mut_proba,G,id2,nb_connections)
    rate = rate_perc*length(id2);
    rate = convert(Int64, round(rate, digits=0));
    weights = nb_connections ./ sum(nb_connections);
    weights = 1 .- weights;
    for i in 1:size(pop,1)
        if (rand(Uniform(0, 1)) < mut_proba)
            c = sample(1:length(id2),Weights(weights),rate, replace = false);
            for j in 1:rate
                b = findall(x-> x!=0,G[id2[c[j]],:]);
                a = rand(1:length(b));
                while (b[a]==pop[i,id2[c[j]]])
                        a = rand(1:length(b));
                end
                pop[i,id2[c[j]]] = b[a];
            end
        end
    end
    return pop;
end

mutation_decreased_weight (generic function with 1 method)

## genetic_algo_normal

In [25]:
function genetic_algo_normal(population_size,select_perc,initialization_perc,mut_proba,cross_proba,
                                        rate_perc,G,id2,type_ctr_connection,nb_iterations,remain_connection_ctr,up_bound,
                                        output_f,output_weight,input_f,input_weight,nb_connections)
    best_ind = zeros(Int,collect(size(G))[1]);
    best_result = zeros(Int,3);
    remain_individuals = population_size - select_perc*population_size;
    remain_individuals= Int64(remain_individuals);
    pop =heuristic_initialization(G,population_size,initialization_perc,id2,nb_connections);
    counter = 0;
    while(counter<nb_iterations)
        y,performance,best_individual = evaluation(G,pop,type_ctr_connection,remain_connection_ctr,
                                                    up_bound,output_f,output_weight,input_f,input_weight);
        if(performance[1]>best_result[1])
            "performance[1] is the highest adequacy degree obtained in the current iteration"
            best_result = performance;
            best_ind = best_individual;
            counter = 0;
        else
            counter += 1;
        end
        select_pop = selection(pop,select_perc,y);
        pop = crossover(select_pop,remain_individuals,cross_proba);
        pop = mutation(pop,rate_perc,mut_proba,G,id2);
    end
    return best_result,best_ind;    
end
    

genetic_algo_normal (generic function with 1 method)

## genetic_algo_increased_mutation_weight

In [26]:
function genetic_algo_increased_mutation_weight(population_size,select_perc,initialization_perc,mut_proba,
                                                cross_proba,rate_perc,G,id2,type_ctr_connection,nb_iterations,
                                                remain_connection_ctr,up_bound,output_f,output_weight,input_f,
                                                input_weight,nb_connections)
    best_ind = zeros(Int,collect(size(G))[1]);
    best_result = zeros(Int,3);
    remain_individuals = population_size - select_perc*population_size;
    remain_individuals= Int64(remain_individuals);
    pop =heuristic_initialization(G,population_size,initialization_perc,id2,nb_connections);
    counter = 0;
    while(counter<nb_iterations)
        y,performance,best_individual = evaluation(G,pop,type_ctr_connection,remain_connection_ctr,
                                                    up_bound,output_f,output_weight,input_f,input_weight);
        if(performance[1]>best_result[1])
            "performance[1] is the highest adequacy degree obtained in the current iteration"
            best_result = performance;
            best_ind = best_individual;
            counter = 0;
        else
            counter += 1;
        end
        select_pop = selection(pop,select_perc,y);
        pop = crossover(select_pop,remain_individuals,cross_proba);
        pop = mutation_increased_weight(pop,rate_perc,mut_proba,G,id2,nb_connections);
    end
    return best_result,best_ind;    
end

genetic_algo_increased_mutation_weight (generic function with 1 method)

## genetic_algo_decreased_mutation_weight 

In [27]:
function genetic_algo_decreased_mutation_weight(population_size,select_perc,initialization_perc,mut_proba,
                                                cross_proba,rate_perc,G,id2,type_ctr_connection,nb_iterations,
                                                remain_connection_ctr,up_bound,output_f,output_weight,input_f,
                                                input_weight,nb_connections)
    best_ind = zeros(Int,collect(size(G))[1]);
    best_result = zeros(Int,3);
    remain_individuals = population_size - select_perc*population_size;
    remain_individuals= Int64(remain_individuals);
    pop =heuristic_initialization(G,population_size,initialization_perc,id2,nb_connections);
    counter = 0;
    while(counter<nb_iterations)
        y,performance,best_individual = evaluation(G,pop,type_ctr_connection,remain_connection_ctr,
                                                    up_bound,output_f,output_weight,input_f,input_weight);
        if(performance[1]>best_result[1])
            "performance[1] is the highest adequacy degree obtained in the current iteration"
            best_result = performance;
            best_ind = best_individual;
            counter = 0;
        else
            counter += 1;
        end
        select_pop = selection(pop,select_perc,y);
        pop = crossover(select_pop,remain_individuals,cross_proba);
        pop = mutation_decreased_weight(pop,rate_perc,mut_proba,G,id2,nb_connections);
    end
    return best_result,best_ind;    
end

genetic_algo_decreased_mutation_weight (generic function with 1 method)

### Read the path of required files

In [17]:
connection_ctr1 ="C:\\Users\\AH262855\\Desktop\\Nouveau dossier\\Data_ML\\perc_initialization\\NbFunctions_5000\\NbCommunications_100\\ComN1N1_F5000_C100_X0_10.csv";
F2 ="C:\\Users\\AH262855\\Desktop\\Nouveau dossier\\Data_ML\\perc_initialization\\NbFunctions_5000\\NbCommunications_100\\functionsWithDomains_F5000_C100_10.csv";


### Initialize the parameters of genetic algorithm

In [18]:
population_size = 100;
select_perc  = 0.5 
initialization_perc = 0.2;
mut_proba = 0.05;
cross_proba = 0.5;
nb_iterations = 20;
rate_perc = 0.02;

### Execution of create_features function and calculate uper bound of study case

In [19]:
"up_bound is the uper bound of the study case
it is calculated by sum the maximum degree of all the functions"
G,id1,id2,type_ctr_connection,remain_connection_ctr,output_f,output_weight,input_f,input_weight,nb_connections = 
                                                                create_features(F2,connection_ctr1);
nb_ctr = size(remain_connection_ctr,1);
up_bound =0;
for j in 1:size(G,1)
    up_bound += maximum(G[j,1:nb_ctr]);
end
up_bound


11077

 Execution 10 times of genetic algoritm with random mutation strategy
 <br>The mean of results obtained is in normal_mut variable

In [31]:
normal_mut = zeros(3);
for i in 1:5
    best_result,best_ind = genetic_algo_normal(population_size,select_perc,initialization_perc,mut_proba,
                                               cross_proba,rate_perc,G,id2,type_ctr_connection,nb_iterations,
                                               remain_connection_ctr,up_bound,output_f,output_weight,input_f,
                                                input_weight,nb_connections)
    normal_mut .+= best_result;
    println("iteration: ",i," ",best_result);
end
println("results of normal mutation: ",normal_mut./5);

iteration: 1 [10567, 0, 0]
iteration: 2 [10578, 0, 0]
iteration: 3 [10575, 0, 0]
iteration: 4 [10556, 0, 0]
iteration: 5 [10567, 0, 0]
results of normal mutation: [10568.6, 0.0, 0.0]


 Execution 10 times of genetic algoritm with first created mutation strategy
 <br>The mean of results obtained is in increased_mut variable

In [32]:
increased_mut = zeros(3);
for i in 1:10 
    best_result,best_ind = genetic_algo_increased_mutation_weight(population_size,select_perc,
                                                                 initialization_perc,mut_proba,cross_proba,rate_perc,G,
                                                                 id2,type_ctr_connection,nb_iterations,remain_connection_ctr
                                                                 ,up_bound,output_f,output_weight,input_f,
                                                                 input_weight,nb_connections)
    increased_mut .+= best_result;
    println("iteration: ",i," ",best_result);
end
println("results of increased mutation strategy: ",increased_mut./10);

iteration: 1 [10566, 0, 0]
iteration: 2 [10568, 0, 0]
iteration: 3 [10586, 0, 0]
iteration: 4 [10578, 0, 0]
iteration: 5 [10571, 0, 0]
iteration: 6 [10582, 0, 0]
iteration: 7 [10575, 0, 0]
iteration: 8 [10570, 0, 0]
iteration: 9 [10588, 0, 0]
iteration: 10 [10570, 0, 0]
results of increased mutation strategy: [10575.4, 0.0, 0.0]


 Execution 10 times of genetic algoritm with second created mutation strategy
 <br>The mean of results obtained is in decreased_mut variable

In [ ]:
decreased_mut = zeros(3);
for i in 1:10 
    best_result,best_ind = genetic_algo_increased_mutation_weight(population_size,select_perc,
                                                                 initialization_perc,mut_proba,cross_proba,rate_perc,G,
                                                                 id2,type_ctr_connection,nb_iterations,remain_connection_ctr
                                                                 ,up_bound,output_f,output_weight,input_f,
                                                                 input_weight,nb_connections)
    decreased_mut .+= best_result;
    println("iteration: ",i," ",best_result);
end
println("results of decreased mutation: ",decreased_mut./10);

iteration: 1 [10585, 0, 0]
iteration: 2 [10585, 0, 0]
iteration: 3 [10577, 0, 0]
iteration: 4 [10585, 0, 0]
iteration: 5 [10575, 0, 0]
iteration: 6 [10586, 0, 0]